# 数据合并与分组

## 1、数据合并

In [1]:
import pandas as pd

staff_df = pd.DataFrame([{'姓名': '张三', '部门': '研发部'},
                        {'姓名': '李四', '部门': '财务部'},
                        {'姓名': '赵六', '部门': '市场部'}])


student_df = pd.DataFrame([{'姓名': '张三', '专业': '计算机'},
                        {'姓名': '李四', '专业': '会计'},
                        {'姓名': '王五', '专业': '市场营销'}])

print(staff_df)
print()
print(student_df)

   姓名   部门
0  张三  研发部
1  李四  财务部
2  赵六  市场部

     专业  姓名
0   计算机  张三
1    会计  李四
2  市场营销  王五


### pd.merge() 通过键拼接列

https://blog.csdn.net/zutsoft/article/details/51498026

In [2]:
pd.merge(staff_df, student_df, how='outer', on='姓名')
# 或者：
# staff_df.merge(student_df, how='outer', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计
2,赵六,市场部,NaN
3,王五,NaN,市场营销


In [3]:
pd.merge(staff_df, student_df, how='inner', on='姓名')
# 或者：
# staff_df.merge(student_df, how='inner', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计


In [4]:
pd.merge(staff_df, student_df, how='left', on='姓名')
# 或者：
# staff_df.merge(student_df, how='left', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计
2,赵六,市场部,NaN


In [5]:
pd.merge(staff_df, student_df, how='right', on='姓名')
# 或者：
# staff_df.merge(student_df, how='right', on='姓名')

,姓名,部门,专业
0,张三,研发部,计算机
1,李四,财务部,会计
2,王五,NaN,市场营销


In [6]:
# 也可以按照索引进行合并
staff_df.set_index('姓名', inplace=True)
student_df.set_index('姓名', inplace=True)

print(staff_df)
print()
print(student_df)
print()

pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

     部门
姓名     
张三  研发部
李四  财务部
赵六  市场部

      专业
姓名      
张三   计算机
李四    会计
王五  市场营销



,部门,专业
姓名,,
张三,研发部,计算机
李四,财务部,会计
王五,NaN,市场营销
赵六,市场部,NaN


In [7]:
# 当数据中列名不同时，使用 left_on、right_on 属性
staff_df.reset_index(inplace=True)  # 重置为默认 index
student_df.reset_index(inplace=True)

staff_df.rename(columns={'姓名':'员工姓名'}, inplace=True)  # 修改 staff_df 的列名

print(staff_df)
print()
print(student_df)

pd.merge(staff_df, student_df, how='outer', left_on='员工姓名', right_on='姓名')

  员工姓名   部门
0   张三  研发部
1   李四  财务部
2   赵六  市场部

   姓名    专业
0  张三   计算机
1  李四    会计
2  王五  市场营销


,员工姓名,部门,姓名,专业
0,张三,研发部,张三,计算机
1,李四,财务部,李四,会计
2,赵六,市场部,NaN,NaN
3,NaN,NaN,王五,市场营销


In [22]:
# 如果两个数据集中包含相同列名（但不是要连接的列），merge() 会自动加后缀作为区分
staff_df['地区'] = ['天津', '北京', '南京']
student_df['地区'] = ['天津', '香港', '深圳']

print(staff_df)
print()
print(student_df)

  员工姓名   部门  地区
0   张三  研发部  天津
1   李四  财务部  北京
2   赵六  市场部  南京

   姓名    专业  地区
0  张三   计算机  天津
1  李四    会计  香港
2  王五  市场营销  深圳


In [23]:
pd.merge(staff_df, student_df, how='outer', left_on='员工姓名', right_on='姓名')

,员工姓名,部门,地区_x,姓名,专业,地区_y
0,张三,研发部,天津,张三,计算机,天津
1,李四,财务部,北京,李四,会计,香港
2,赵六,市场部,南京,NaN,NaN,NaN
3,NaN,NaN,NaN,王五,市场营销,深圳


In [24]:
# 也可以指定后缀名称
pd.merge(staff_df, student_df, how='outer', left_on='员工姓名', right_on='姓名', suffixes=('(公司)', '(家乡)'))

,员工姓名,部门,地区(公司),姓名,专业,地区(家乡)
0,张三,研发部,天津,张三,计算机,天津
1,李四,财务部,北京,李四,会计,香港
2,赵六,市场部,南京,NaN,NaN,NaN
3,NaN,NaN,NaN,王五,市场营销,深圳


In [27]:
# 指定多列进行合并，找出工作地址和家乡地址相同的人
pd.merge(staff_df, student_df, how='inner', left_on=['员工姓名', '地区'], right_on=['姓名', '地区'])

,员工姓名,部门,地区,姓名,专业
0,张三,研发部,天津,张三,计算机


In [29]:
# apply() 的使用
# 获取姓
staff_df['员工姓名'].apply(lambda x: x[0])

0    张
1    李
2    赵
Name: 员工姓名, dtype: object

In [30]:
# 获取名
staff_df['员工姓名'].apply(lambda x: x[1:])

0    三
1    四
2    六
Name: 员工姓名, dtype: object

In [36]:
# 结果合并
staff_df['姓'] = list(staff_df['员工姓名'].apply(lambda x: x[0]))
staff_df['名'] = list(staff_df['员工姓名'].apply(lambda x: x[1:]))

student_df['姓'] = list(map(lambda x: x[0], student_df['姓名']))
student_df['名'] = list(map(lambda x: x[1:], student_df['姓名']))

staff_df

,员工姓名,部门,地区,姓,名
0,张三,研发部,天津,张,三
1,李四,财务部,北京,李,四
2,赵六,市场部,南京,赵,六


## 2、数据分组

In [37]:
report_data = pd.read_csv('./2015.csv')
report_data.head()

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204
3,Norway,Western Europe,4,7.522,0.03880,1.45900,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176


### groupby()

In [39]:
grouped = report_data.groupby('Region')

print(grouped)
print()
print(type(grouped))


<class 'pandas.core.groupby.DataFrameGroupBy'>


In [40]:
grouped['Happiness Score'].mean()

Region
Australia and New Zealand          7.285000
Central and Eastern Europe         5.332931
Eastern Asia                       5.626167
Latin America and Caribbean        6.144682
Middle East and Northern Africa    5.406900
North America                      7.273000
Southeastern Asia                  5.317444
Southern Asia                      4.580857
Sub-Saharan Africa                 4.202800
Western Europe                     6.689619
Name: Happiness Score, dtype: float64

In [41]:
grouped.size()

Region
Australia and New Zealand           2
Central and Eastern Europe         29
Eastern Asia                        6
Latin America and Caribbean        22
Middle East and Northern Africa    20
North America                       2
Southeastern Asia                   9
Southern Asia                       7
Sub-Saharan Africa                 40
Western Europe                     21
dtype: int64

In [42]:
# 迭代 groupby 对象
for group, frame in grouped:
    print(group, frame)

Australia and New Zealand        Country                     Region  Happiness Rank  Happiness Score  \
8  New Zealand  Australia and New Zealand               9            7.286   
9    Australia  Australia and New Zealand              10            7.284   

   Standard Error  Economy (GDP per Capita)   Family  \
8         0.03371                   1.25018  1.31967   
9         0.04083                   1.33358  1.30923   

   Health (Life Expectancy)  Freedom  Trust (Government Corruption)  \
8                   0.90837  0.63938                        0.42922   
9                   0.93156  0.65124                        0.35637   

   Generosity  Dystopia Residual  
8     0.47501            2.26425  
9     0.43562            2.26646  
Central and Eastern Europe                     Country                      Region  Happiness Rank  \
30           Czech Republic  Central and Eastern Europe              31   
43               Uzbekistan  Central and Eastern Europe              44   